In [3]:
# import needed libraries
import numpy as np
import os
import matplotlib.pyplot as plt
from google.colab import drive
import matplotlib.image as mpimg
import pandas as pd
from IPython import display
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import cv2
from zipfile import ZipFile 
from pathlib import Path
import sys
import tensorflow as tf
from PIL import Image
from PIL import ImageChops
import zipfile
from zipfile import ZipFile
from google.colab.patches import cv2_imshow

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
def read_pgm(pgmf):
    """Return a raster of integers from a PGM as a list of lists."""
    pgmf.readline()
    (width, height) = [int(i) for i in pgmf.readline().split()]
    depth = int(pgmf.readline())
    assert depth <= 255

    raster = []
    for y in range(width):
        row = []
        for y in range(height):
            row.append(ord(pgmf.read(1)))
        raster.append(row)
    return raster

In [12]:
path_folder = '/content/drive/My Drive/orl_dataset'
files = os.listdir(path_folder)
files.remove(files[0])
files.remove(files[0])
print(len(files))
print(files)

40
['s1', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's2', 's20', 's21', 's22', 's23', 's24', 's25', 's26', 's27', 's28', 's29', 's3', 's30', 's31', 's32', 's33', 's34', 's35', 's36', 's37', 's38', 's39', 's4', 's40', 's5', 's6', 's7', 's8', 's9']


In [13]:
data_matrix = np.zeros((400,10304))
labels = np.zeros((400,))
k = 0
for i in range(len(files)):
  path_file = path_folder + '/' + files[i]
  pgms = os.listdir(path_file)
  name = files[i]
  name = name.replace(name[0],'')
  for j in range(len(pgms)):
    f = open(path_file+'/'+pgms[j], 'rb')
    r = read_pgm(f)
    r = np.array(r)
    #r = np.true_divide(r, 255)
    r = r.reshape(r.shape[0]*r.shape[1],)
    data_matrix[k] = r
    labels[k] = int(name)
    k = k+1
print(data_matrix.shape)
print(labels.shape)

(400, 10304)
(400,)


**Split 50 %** **for train and 50% for test**

In [14]:
x_train = np.zeros((200,10304))
y_train = np.zeros((200,))
x_test = np.zeros((200,10304))
y_test = np.zeros((200,))
train = 0
test = 0
for i in range(data_matrix.shape[0]):
  if (i % 2 != 0):
    x_train[train] = data_matrix[i]
    y_train[train] = labels[i]
    train = train+1
  else:
    x_test[test] = data_matrix[i]
    y_test[test] = labels[i]
    test = test+1
print(x_train.shape, ' ', y_train.shape,' ', x_test.shape, ' ', y_test.shape)

(200, 10304)   (200,)   (200, 10304)   (200,)


**Split 90% for train and 10% for test**

In [15]:
x_train = np.zeros((360,10304))
y_train = np.zeros((360,))
x_test = np.zeros((40,10304))
y_test = np.zeros((40,))
train = 0
test = 0
var = 0
for i in range(data_matrix.shape[0]):
  if (var <= 8):
    x_train[train] = data_matrix[i]
    y_train[train] = labels[i]
    train = train+1
  else:
    x_test[test] = data_matrix[i]
    y_test[test] = labels[i]
    test = test+1
  var = var + 1
  if (var == 10):
    var = 0
print(x_train.shape, ' ', y_train.shape,' ', x_test.shape, ' ', y_test.shape)

(360, 10304)   (360,)   (40, 10304)   (40,)


**Implement Naive Bayes**

In [23]:
#Naive bayes
def naive_bayes(D, Y):
  num_classes = int(np.max(Y))
  i = 0
  n = D.shape[0]
  s = 0
  pcs = np.zeros((num_classes,))
  all_mean = np.zeros((num_classes,D.shape[1]))
  all_segma = np.zeros((num_classes,D.shape[1],D.shape[1]))
  while (i< (Y.shape[0])):
    segma_class = np.zeros((D.shape[1],))
    dclass = D[i:i+5,:]
    ni = dclass.shape[0]
    pci = ni / n
    pcs[s] = pci
    mean_class = np.mean(dclass, axis=0)
    all_mean[s] = mean_class
    Z = dclass - mean_class.transpose()
    for j in range(D.shape[1]):
      segma_class[j] = np.dot(Z[:,j].transpose(), Z[:,j])
    all_segma[s] = segma_class
    s = s+1
    i = i+5
  return pcs, all_mean, all_segma

**Classifie X**

In [47]:
import math
def testing(x, mean, pc, segma):
  ex = 0
  cal = 1
  for i in range(segma.shape[0]):
    ex = ex + np.power(x[i] - mean[i], 2) / (2 * segma[0,i])
    cal = cal * (1/np.sqrt(2*22/7)) * (1/np.sqrt(segma[0,i]))
  ex = math.exp(-ex)
  yhat = cal * ex
  #ex = np.sum(np.power(x-mean,2) / (2*segma))
  #ex = math.exp(-ex)
  #det = np.power(np.sqrt(2*(22/7)), segma.shape[0]) *  np.sqrt(np.prod(segma))
  #det = 1/det
  #yhat = det * ex
  return yhat

In [28]:
#PCA
def pca(D, alpha):
  mean = np.mean(D,axis = 0)
  Z = D - mean.transpose()
  n = Z.shape[0]
  covariance = (np.dot(Z.transpose(), Z)) / n
  eig_values, eig_vectors = np.linalg.eigh(covariance)
  eig_values = np.diag(eig_values[::-1])
  eig_vectors = eig_vectors[:,::-1]
  eig_values_sum = np.sum(eig_values)
  thresold = alpha * eig_values_sum
  fraction = eig_values[0][0]
  i = 1
  while(fraction < thresold):
    fraction += eig_values[i][i]
    i = i + 1
  projection_matrix = eig_vectors[:, :i]
  return projection_matrix

In [29]:
alpha = np.array([0.8,0.85,0.9,0.95])

In [ ]:
projection_matrix = pca(x_train, alpha[0])
x_train = np.dot(x_train, projection_matrix)
x_test = np.dot(x_test, projection_matrix)
pcs, all_mean, all_segma = naive_bayes(x_train, y_train)
prediction = np.zeros((x_test.shape[0],))
for j in range(x_test.shape[0]):
  ymax = 0
  class_num = 0
  x = x_test[j]
  y = y_test[j]
  for i in range(all_segma.shape[0]):
    yhat = testing(x, all_mean[i], pcs[i], all_segma[i])
    if (ymax < yhat):
      ymax = yhat
      class_num = i+1
  prediction[j] = class_num
print("Prediction:", prediction)
print("Labels:",y_test)

In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, prediction))

**using library for naive bayes in python**

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_train, y_train)
y_pred = gnb.predict(x_test)
print("Prediction:",y_pred)
print("Label:", y_test)

**Accuracy for naive bayes**

In [39]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8125


**Naive Bayes with PCA**

In [16]:
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
projection_matrix = pca(x_train[:40,:], alpha[0])
x_train = np.dot(x_train[:40,:], projection_matrix)
x_test = np.dot(x_test[:5,:], projection_matrix)
gnb = GaussianNB()
gnb.fit(x_train, y_train[0:40,])
y_pred = gnb.predict(x_test)
print("Prediction:",y_pred)
print("Label:", y_test[0:5,])

Prediction: [ 1. 10. 11. 12. 12.]
Label: [ 1. 10. 11. 12. 13.]


**Accuracy for naive bayes with PCA**

In [17]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test[0:5,], y_pred))

Accuracy: 0.8


**Decision Tree**

In [10]:
def decision_tree(D, eita = 10, bay = 1, labels):
  tree = {}
  n = D.shape[0]
  all_class = np.zeros((40, 10, 10304))
  s = 0
  purity = 0
  for i in range(D.shape[0]):
    all_class[s] = D[i:i+10,:]
    if (purity < all_class[s].shape[0]/n):
      purity = all_class[s].shape[0]/n
    s = s+1
    i = i+10
  if (n <= eita or purity >= bay):
    tree[D] = purity
    return
  split_point = 0
  score_star = 0
  column_number = 0
  Dy = []
  Dn = []
  labely = []
  labeln = []
  for j in range(D.shape[1]):
    (v, score) = evaluate_numeric_attribute(D, D[:,j], labels)
    if (score_star < score):
      score_star = score
      split_point = v
      column_number = j
  index_y = np.where(D[:,j] <= split_point)[0]
  for i in range(index_y.shape[0]):
    Dy.append(D[index_y[i]])
    labely.append(labels[index_y[i]])
  index_n = np.where(D[:,j] > split_point)[0] 
  for i in range(index_n.shape[0]):
    Dy.append(D[index_n[i]])
    labeln.append(labels[index_n[i]])
  labely = np.array(labely)
  labeln = np.array(labeln)
  Dy = np.array(Dy)
  Dn = np.array(Dn)
  tree[D] = np.array((Dy, Dn))
  decision_tree(Dy, eita, by, labely)
  decision_tree(Dn, eita, by, labeln)

**Evaluate numeric attributes**

In [ ]:
def evaluate_numeric_attribute(D, xj, labels):
  label_set = set (labels)
  nis = np.zeros((len(label_set,)))
  l2 = 0
  for l in label_set:
    nis[l2] = np.where(labels == l)[0].shape[0]
    l2 = l2+1

**Using the library**

In [17]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train)
y_pred = clf.predict(x_test) 
print("Prediction:",y_pred)
print("Labels:", y_test)

Prediction: [ 1. 31. 24.  8.  1. 14. 38. 14. 21. 21. 11. 39. 11. 11. 11. 12. 37. 26.
  2. 10. 13. 13. 13. 37. 13. 14. 14. 14. 14.  2. 15. 15. 32. 19. 19. 24.
 16.  1. 24.  3.  7.  7. 17. 23. 21.  6.  6.  6. 18. 18. 32. 19.  3. 32.
 37. 32. 32.  2.  2. 32. 21. 20. 30. 38. 20. 30. 21. 21. 30. 21.  4. 22.
 24. 30. 35. 23. 23. 29. 22. 23. 24. 24. 24. 24. 19. 25. 31. 25.  5.  4.
 28. 13. 13. 37. 26. 27. 27.  2. 27. 32. 28. 16.  3. 28. 28. 33. 33. 21.
 29. 21. 37. 21. 26.  3.  3. 21. 30. 21. 30. 30. 31. 31. 31. 31. 31. 32.
 17.  8. 32.  8. 33. 39. 39. 30. 33. 34. 33. 34. 31. 34. 23.  1. 31. 27.
 15.  7. 21. 36. 36. 36. 37. 37. 24. 24. 28. 38. 38. 20. 26. 23. 31. 31.
 38. 33. 29.  4.  4.  4. 17.  4.  5. 17. 40. 18. 13.  5.  5. 40.  5. 26.
  6. 33.  6.  6.  6.  7. 21.  7. 39. 29.  8.  8.  8.  8.  8.  9.  4.  9.
  9.  9.]
Labels: [ 1.  1.  1.  1.  1. 10. 10. 10. 10. 10. 11. 11. 11. 11. 11. 12. 12. 12.
 12. 12. 13. 13. 13. 13. 13. 14. 14. 14. 14. 14. 15. 15. 15. 15. 15. 16.
 16. 16. 16. 16. 17. 

**PCA with Decision Tree**

In [34]:
from sklearn import tree
proj = pca(x_train, alpha[0])
x_train2 = np.dot(x_train[0:150,:], proj)
x_test2 = np.dot(x_test[0:150,:], proj)
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train2, y_train[0:150,])
y_pred = clf.predict(x_test2) 
print("Prediction:",y_pred)
print("Labels:", y_test[0:150,])

Prediction: [16. 13. 13. 13.  1. 36. 36. 10. 36. 10. 11. 10. 19. 10. 21.  1. 12. 12.
  2. 12. 13. 13. 13. 13. 13. 14. 14. 14. 14. 14. 17. 15. 15. 19. 15.  3.
 35. 16. 17. 16. 35. 17. 17. 36. 23. 13. 36. 36.  2. 18. 19. 19. 36. 14.
 36. 17.  2.  2. 25.  2. 21. 30. 24. 24. 23. 30. 21. 15. 15. 21. 22. 22.
 22. 22. 22. 23. 23. 23. 23. 23. 24. 24. 24. 11. 11. 25.  3. 25. 25. 25.
 32. 26. 26. 17. 26. 27. 27. 27. 17. 17. 28. 32. 28. 26. 26. 29. 29. 29.
 29. 29. 12. 15.  3. 17.  3. 30. 30. 30. 22. 22. 15. 31. 31. 20. 15. 32.
  1. 25. 32. 32. 22. 33. 11. 15. 33. 34. 34. 34. 34. 34. 17. 17. 17. 17.
 17. 32. 36. 36. 35. 36.]
Labels: [ 1.  1.  1.  1.  1. 10. 10. 10. 10. 10. 11. 11. 11. 11. 11. 12. 12. 12.
 12. 12. 13. 13. 13. 13. 13. 14. 14. 14. 14. 14. 15. 15. 15. 15. 15. 16.
 16. 16. 16. 16. 17. 17. 17. 17. 17. 18. 18. 18. 18. 18. 19. 19. 19. 19.
 19.  2.  2.  2.  2.  2. 20. 20. 20. 20. 20. 21. 21. 21. 21. 21. 22. 22.
 22. 22. 22. 23. 23. 23. 23. 23. 24. 24. 24. 24. 24. 25. 25. 25. 25. 25.
 26. 

In [35]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test[0:150,], y_pred))

Accuracy: 0.5466666666666666


In [36]:
#number of tree = 300
proj = pca(x_train, alpha[0])
x_train2 = np.dot(x_train[0:100,:], proj)
x_test2 = np.dot(x_test[0:100,:], proj)

In [43]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=300)
clf.fit(x_train2,y_train[0:100,])
y_pred=clf.predict(x_test2)
print("Prediction:",y_pred)
print("Labels:", y_test[0:100,])

Prediction: [16.  1.  1.  1.  1. 10. 10. 10. 10. 10. 11. 11. 11. 11. 11. 12. 12. 12.
 12. 12. 13. 13. 13. 13. 13. 14. 14. 14. 14. 14. 15. 15. 15. 15. 15. 16.
 16. 16. 17. 16. 17. 17. 17. 17. 17. 18. 18. 18. 18. 18. 19. 19. 19. 19.
 19.  2.  2.  2.  2.  2. 20. 20. 20. 22. 20. 21. 21. 21. 21. 21. 22. 22.
 22. 22. 22. 23. 23. 23. 23. 23. 24. 24. 24. 24. 24. 25. 25. 25. 25. 25.
 26. 26. 26. 26. 26. 27. 27. 27. 27. 27.]
Labels: [ 1.  1.  1.  1.  1. 10. 10. 10. 10. 10. 11. 11. 11. 11. 11. 12. 12. 12.
 12. 12. 13. 13. 13. 13. 13. 14. 14. 14. 14. 14. 15. 15. 15. 15. 15. 16.
 16. 16. 16. 16. 17. 17. 17. 17. 17. 18. 18. 18. 18. 18. 19. 19. 19. 19.
 19.  2.  2.  2.  2.  2. 20. 20. 20. 20. 20. 21. 21. 21. 21. 21. 22. 22.
 22. 22. 22. 23. 23. 23. 23. 23. 24. 24. 24. 24. 24. 25. 25. 25. 25. 25.
 26. 26. 26. 26. 26. 27. 27. 27. 27. 27.]


In [44]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test[0:100,], y_pred))

Accuracy: 0.97
